## Berry, Levinsohn, and Pakes (1995, 1999) Replication

Obj: Replicate Table 8 from Colton and Gortmaker 2020 (page 1149)

In [1]:
import pyblp
import numpy as np
import pandas as pd

pyblp.options.digits = 2
pyblp.options.verbose = False
pyblp.options.weights_tol = np.inf   # To disabled warning of sum(weight)!=1 in the problem class

### Here we create the problem class

Important:
- The demand-side linear characteristics (X1) is the first line of the product formulations
- The demand-side non-linear characteristics (X2) is the second line of the product formulations
- The supply-side characteristics (X3) is the third line of the product formulations
- The agent formulation adds the price interaction from demographic information
    - I() indicates a mathematical operation
- costs_type means the type of function of the marginal cost (for the X3)

In [2]:
problem = pyblp.Problem(
    product_formulations = (
        pyblp.Formulation('1 + hpwt + air + mpd + space'),
        pyblp.Formulation('1 + prices + hpwt + air + mpd + space'),
        pyblp.Formulation('1 + log(hpwt) + air + log(mpg) + log(space) + trend')
    ),
    agent_formulation = pyblp.Formulation('0 + I(1 / income)'),
    costs_type = 'log',
    product_data = pd.read_csv(pyblp.data.BLP_PRODUCTS_LOCATION), 
    agent_data = pd.read_csv(pyblp.data.BLP_AGENTS_LOCATION)
)
problem

Dimensions:
 T    N     F    I     K1    K2    K3    D    MD    MS 
---  ----  ---  ----  ----  ----  ----  ---  ----  ----
20   2217  26   4000   5     6     6     1    13    18 

Formulations:
       Column Indices:            0          1       2       3          4         5  
-----------------------------  --------  ---------  ----  --------  ----------  -----
 X1: Linear Characteristics       1        hpwt     air     mpd       space          
X2: Nonlinear Characteristics     1       prices    hpwt    air        mpd      space
X3: Log Cost Characteristics      1      log(hpwt)  air   log(mpg)  log(space)  trend
       d: Demographics         1/income                                              

From the dimension table:
|T|N|F|I|K1|K2|K3|D|MD|MS|
|-|-|-|-|--|--|--|-|--|--|
|Number of markets|Number of products across all markets|Number of firms across all markets|Number of agents across all markets|Number of demand-side linear product characteristics|Number of demand-side nonlinear product characteristics|Number of supply-side product characteristics|Number of demographic variable|Number of demand-side instruments, which are typically the number of excluded demand-side instruments plus the number of exogenous demand-side linear product characteristics|Number of supply-side instruments, which are typically the number of excluded supply-side instruments plus the number of exogenous supply-side linear product characteristics|

### First we solve replicating BLP's work

Explanation
- sigma is a matrix that fixed at zero or at starting values of the (lower-triangular Cholesky root) covariance matrix of the nonlinear characteristics (unobserved heterogeneity)
    - Rows and columns correspond to columns of the variables of X2
- pi is a matrix that fixed at zero or at starting values the parameters of how agents preferences change with demographics (observed heterogeneity)
    - Rows correspond to the same product characteristic of sigma and the columns are the columns of the demographics

In [5]:
initial_sigma = np.diag([3.612, 0, 4.628, 1.818, 1.050, 2.056])
initial_pi = np.array([[0],[-43.501],[0],[0],[0],[0]])

In [6]:
results_replication = problem.solve(
    initial_sigma,
    initial_pi,
    initial_update=True,           # update the weight matrix to starting values before first GMM
    costs_bounds=(0.001, None),    # bounds for costs_type (since is logarithmic having a lb avoid nonpositive costs)
    W_type='clustered',            # update the weight matrix cluster by automovil model
    se_type='clustered',
)
results_replication

Problem Results Summary:
GMM   Objective    Projected    Reduced Hessian  Reduced Hessian  Clipped  Clipped  Weighting Matrix  Covariance Matrix
Step    Value    Gradient Norm  Min Eigenvalue   Max Eigenvalue   Shares    Costs   Condition Number  Condition Number 
----  ---------  -------------  ---------------  ---------------  -------  -------  ----------------  -----------------
 2    +5.0E+02     +2.3E-06        +4.9E-01         +5.1E+02         0        0         +4.2E+09          +3.8E+08     

Cumulative Statistics:
Computation  Optimizer  Optimization   Objective   Fixed Point  Contraction
   Time      Converged   Iterations   Evaluations  Iterations   Evaluations
-----------  ---------  ------------  -----------  -----------  -----------
 00:03:03       No           58           167         48339       148305   

Nonlinear Coefficient Estimates (Robust SEs Adjusted for 999 Clusters in Parentheses):
Sigma:      1        prices      hpwt        air         mpd        space     |

In [7]:
# Calculate own elasticities
e_replication = results_replication.compute_elasticities()
e_replication_means = results_replication.extract_diagonal_means(e_replication)

# Calculate Markup
costs = results_replication.compute_costs()
markups_replication = results_replication.compute_markups(costs=costs)

------------------------
### Second we replicate the experiment with best practice

In [8]:
instrument_results = results_replication.compute_optimal_instruments(method='approximate')
optimal_instrument_problem = instrument_results.to_problem()

results_bestpractice = optimal_instrument_problem.solve(
    sigma = results_replication.sigma,
    pi = results_replication.pi,
    initial_update=True,           # update the weight matrix to starting values before first GMM
    costs_bounds=(0.001, None),    # bounds for costs_type (since is logarithmic having a lb avoid nonpositive costs)
    W_type='clustered',            # update the weight matrix cluster by automovil model
    se_type='clustered',
)
results_bestpractice

Problem Results Summary:
GMM   Objective    Projected    Reduced Hessian  Reduced Hessian  Clipped  Clipped  Weighting Matrix  Covariance Matrix
Step    Value    Gradient Norm  Min Eigenvalue   Max Eigenvalue   Shares    Costs   Condition Number  Condition Number 
----  ---------  -------------  ---------------  ---------------  -------  -------  ----------------  -----------------
 2    +1.4E+02     +1.7E-06        +3.5E-01         +1.1E+02         0        0         +8.1E+07          +3.1E+07     

Cumulative Statistics:
Computation  Optimizer  Optimization   Objective   Fixed Point  Contraction
   Time      Converged   Iterations   Evaluations  Iterations   Evaluations
-----------  ---------  ------------  -----------  -----------  -----------
 00:02:43       No           49           146         33470       102837   

Nonlinear Coefficient Estimates (Robust SEs Adjusted for 999 Clusters in Parentheses):
Sigma:      1        prices      hpwt        air         mpd        space     |

In [9]:
# Calculate own elasticities
e_bestpractice = results_bestpractice.compute_elasticities()
e_bestpractice_means = results_bestpractice.extract_diagonal_means(e_bestpractice)

# Calculate Markup
costs = results_bestpractice.compute_costs()
markups_bestpractice = results_bestpractice.compute_markups(costs=costs)

--------------------------
### Table of Results

In [10]:
r = results_replication
b = results_bestpractice

pd.options.display.float_format = '{:,.3f}'.format

In [20]:
data = np.matrix([
    [-7.061, r.beta[0,0],     b.beta[0,0]],
    [0.941,  r.beta_se[0,0],  b.beta_se[0,0]],
    [2.883,  r.beta[1,0],     b.beta[1,0]],
    [2.019,  r.beta_se[1,0],  b.beta_se[1,0]],
    [1.521,  r.beta[2,0],     b.beta[2,0]],
    [0.891,  r.beta_se[2,0],  b.beta_se[2,0]],
    [-0.122, r.beta[3,0],     b.beta[3,0]],
    [0.320,  r.beta_se[3,0],  b.beta_se[3,0]],
    [3.460,  r.beta[4,0],     b.beta[4,0]],
    [0.610,  r.beta_se[4,0],  b.beta_se[4,0]],
    [3.612,  r.sigma[0,0],    b.sigma[0,0]],
    [1.485,  r.sigma_se[0,0], b.sigma_se[0,0]],
    [4.628,  r.sigma[2,2],    b.sigma[2,2]],
    [1.885,  r.sigma_se[2,2], b.sigma_se[2,2]],
    [1.818,  r.sigma[3,3],    b.sigma[3,3]],
    [1.695,  r.sigma_se[3,3], b.sigma_se[3,3]],
    [1.050,  r.sigma[4,4],    b.sigma[4,4]],
    [0.272,  r.sigma_se[4,4], b.sigma_se[4,4]],
    [2.056,  r.sigma[5,5],    b.sigma[5,5]],
    [0.585,  r.sigma_se[5,5], b.sigma_se[5,5]],
    [43.501, r.pi[2,0],       b.pi[2,0]],
    [6.427,  r.pi_se[2,0],    b.pi_se[2,0]],
    [0.952,  r.gamma[0,0],    b.gamma[0,0]],
    [0.194,  r.gamma_se[0,0], b.gamma_se[0,0]],
    [0.477,  r.gamma[1,0],    b.gamma[1,0]],
    [0.056,  r.gamma_se[1,0], b.gamma_se[1,0]],
    [0.619,  r.gamma[2,0],    b.gamma[2,0]],
    [0.038,  r.gamma_se[2,0], b.gamma_se[2,0]],
    [-0.415, r.gamma[3,0],    b.gamma[3,0]],
    [0.055,  r.gamma_se[3,0], b.gamma_se[3,0]],
    [-0.046, r.gamma[4,0],    b.gamma[4,0]],
    [0.081,  r.gamma_se[4,0], b.gamma_se[4,0]],
    [0.019,  r.gamma[5,0],    b.gamma[5,0]],
    [0.002,  r.gamma_se[5,0], b.gamma_se[5,0]],
    [np.nan, np.mean(e_replication_means), np.mean(e_bestpractice_means)],
    [np.nan, np.mean(markups_replication), np.mean(markups_bestpractice)],
    [np.nan, r.objective[0,0]/problem.N,   b.objective[0,0]/problem.N],
    [np.nan, r.objective[0,0],             b.objective[0,0]]
])
indexes = [np.array(['Means','Means','Means','Means','Means','Means','Means','Means','Means','Means',
                     'Standard Deviations','Standard Deviations','Standard Deviations','Standard Deviations',
                     'Standard Deviations','Standard Deviations','Standard Deviations','Standard Deviations',
                     'Standard Deviations','Standard Deviations',
                     'Term on price','Term on price',
                     'Supply-side terms','Supply-side terms','Supply-side terms','Supply-side terms',
                     'Supply-side terms','Supply-side terms','Supply-side terms','Supply-side terms',
                     'Supply-side terms','Supply-side terms','Supply-side terms','Supply-side terms',
                     'Mean own-price elasticity','Mean markup','GMM objective','GMM objective scaled by N']),
           np.array(['Constant','constant.se','HP/weight','hp/weight.se','Air','air.se','MP$','mpd.se','Size','size.se',
                     'Constant','constant.se','HP/weight','hp/weight.se',
                     'Air','air.se','MP$','mpd.se',
                     'Size','size.se',
                     'ln(y-p)','ln(y-p).se',
                     'Constant','constant.se','ln(HP/weight)','ln(hp/weight).se',
                     'Air','air.se','ln(MPG)','ln(mpg).se',
                     'ln(Size)','ln(size).se','Trend','trend.se',
                     '','','',''])]

nevo_table = pd.DataFrame(data, columns=['BLP_Original','Replication','Best Practice'],
                         index=indexes)
nevo_table

BLP_Original  Replication  \
Means                     Constant                -7.061       -7.284   
                          constant.se              0.941        2.807   
                          HP/weight                2.883        3.460   
                          hp/weight.se             2.019        1.415   
                          Air                      1.521       -0.999   
                          air.se                   0.891        2.101   
                          MP$                     -0.122        0.421   
                          mpd.se                   0.320        0.250   
                          Size                     3.460        4.178   
                          size.se                  0.610        0.658   
Standard Deviations       Constant                 3.612        2.025   
                          constant.se              1.485        6.066   
                          HP/weight                4.628        6.100   
                          hp/weight.se             1.885        2.201   
                          Air                      1.818        3.956   
                          air.se                   1.695        2.110   
                          MP$                      1.050        0.254   
                          mpd.se                   0.272        0.549   
                          Size                     2.056        1.908   
                          size.se                  0.585        1.108   
Term on price             ln(y-p)                 43.501        0.000   
                          ln(y-p).se               6.427          NaN   
Supply-side terms         Constant                 0.952        2.760   
                          constant.se              0.194        0.116   
                          ln(HP/weight)            0.477        0.897   
                          ln(hp/weight).se         0.056        0.072   
                          Air                      0.619        0.423   
                          air.se                   0.038        0.087   
                          ln(MPG)                 -0.415       -0.525   
                          ln(mpg).se               0.055        0.073   
                          ln(Size)                -0.046       -0.261   
                          ln(size).se              0.081        0.210   
                          Trend                    0.019        0.027   
                          trend.se                 0.002        0.003   
Mean own-price elasticity                            NaN       -3.904   
Mean markup                                          NaN        0.316   
GMM objective                                        NaN        0.224   
GMM objective scaled by N                            NaN      497.336   

                                            Best Practice  
Means                     Constant                 -6.633  
                          constant.se               0.793  
                          HP/weight                 2.258  
                          hp/weight.se              0.820  
                          Air                       0.176  
                          air.se                    0.224  
                          MP$                       0.268  
                          mpd.se                    0.164  
                          Size                      3.311  
                          size.se                   0.495  
Standard Deviations       Constant                  1.591  
                          constant.se               1.926  
                          HP/weight                 1.941  
                          hp/weight.se              2.340  
                          Air                       1.370  
                          air.se                    0.434  
                          MP$                       0.105  
                          mpd.se                    0.430  
                          Size                      0.96